In [ ]:
# ----------------------------
# 1. IMPORT LIBRARIES
# ----------------------------
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# ----------------------------
# 2. LOAD AND PREPROCESS MNIST
# ----------------------------
# Load data
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten images to vectors
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print("✅ Data shape:", x_train.shape, x_test.shape)

In [ ]:
# ----------------------------
# 3. BUILD AUTOENCODER
# ----------------------------
# Input layer
input_img = Input(shape=(784,))

# Encoder
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu', name='latent_space')(encoded)

# Decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

# Model
autoencoder = Model(input_img, decoded)

# Display model summary
autoencoder.summary()

In [ ]:
# ----------------------------
# 4. TRAIN AUTOENCODER
# ----------------------------
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

history = autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test),
    verbose=1
)

In [ ]:
# ----------------------------
# 5. PLOT TRAINING & VALIDATION LOSS
# ----------------------------
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ----------------------------
# 6. RECONSTRUCT TEST IMAGES
# ----------------------------
# Reconstruct
decoded_imgs = autoencoder.predict(x_test)

# Plot original vs reconstructed
n = 10  # number of digits to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
    
    # Reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.suptitle('Original (Top) vs Reconstructed (Bottom)', fontsize=16)
plt.tight_layout()
plt.show()